# Inicialização

In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.optim as optim
from torcheval.metrics.functional import multiclass_f1_score

import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import copy
from tqdm import tqdm
# from tqdm import tqdm_notebook as tqdm
import time
import os
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
tarefas = np.array(['climbingdown', 'climbingup', 'jumping', 'lying', 'running', 'sitting', 'standing', 'walking'])
SAC = np.array(['sitting', 'standing', 'walking', 'climbingup', 'climbingdown', 'running'])
posicao = np.array(['chest', 'forearm', 'head', 'shin', 'thigh', 'upperarm', 'waist'])

In [4]:
pasta = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\Dataset\\realworld views\\'
D = [None] * 7
D[0] = pd.read_csv(pasta+posicao[0]+'_nova50hz.csv')
D[1] = pd.read_csv(pasta+posicao[1]+'_nova50hz.csv')
D[2] = pd.read_csv(pasta+posicao[2]+'_nova50hz.csv')
D[3] = pd.read_csv(pasta+posicao[3]+'_nova50hz.csv')
D[4] = pd.read_csv(pasta+posicao[4]+'_nova50hz.csv')
D[5] = pd.read_csv(pasta+posicao[5]+'_nova50hz.csv')
D[6] = pd.read_csv(pasta+posicao[6]+'_nova50hz.csv')

In [5]:
class RealWorldDataset(Dataset):
    def __init__(self, X, y):
        # convert into PyTorch tensors and remember them
        self.X = X.clone().detach().to(device)
        self.y = y.clone().detach().to(device)
        # self.X = torch.tensor(X, dtype=torch.float32, device=device)
        # self.y = torch.tensor(y, dtype=torch.float32, device=device)

    def __len__(self):
        # this should return the size of the dataset
        return len(self.X)

    def __getitem__(self, idx):
        # this should return one sample from the dataset
        features = self.X[idx]
        target = self.y[idx]
        return features, target

In [92]:
posi = 0
N = D[posi].shape[0]
X = D[posi].iloc[:,:450].values.reshape(N,3,150)
y = D[posi].iloc[:,-3].values
dataset = RealWorldDataset(X, y)

In [339]:
px.histogram(D[1]['activity code'])

In [93]:
batch_size = 8
trainLoader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
loader = DataLoader(list(zip(X_train, y_train)), shuffle=True, batch_size=16)

In [94]:
X_batch, y_batch = next(iter(trainLoader))
print(X_batch.shape, y_batch.shape)

torch.Size([8, 3, 150]) torch.Size([8])


In [10]:
aux = X_batch.cpu().detach().numpy()
px.line(aux[4,:,:].transpose())

In [28]:
conv1 = nn.Conv1d(3, 16, 3, device=device)
y = conv1(X_batch).cpu().detach().numpy()
px.line(y[4,:,:].transpose())

# Definição do modelo de Chang

In [257]:
model = nn.Sequential(
    nn.Conv1d(in_channels=3, out_channels=16, kernel_size=3, device=device),
    nn.LeakyReLU(0.3),
    nn.InstanceNorm1d(16, device=device),
    nn.Dropout(p=0.2),

    nn.Conv1d(in_channels=16, out_channels=100, kernel_size=5, stride=1, device=device),
    nn.LeakyReLU(0.3),
    # nn.InstanceNorm1d(100, device=device),

    nn.AvgPool1d(kernel_size=144),
    nn.Flatten(start_dim=1),
    nn.Linear(in_features=100, out_features=8, device=device)
)

In [248]:
glap = nn.Sequential(nn.AvgPool1d(kernel_size=144))

In [6]:
class ChangEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv1d(in_channels=3, out_channels=16, kernel_size=3, device=device)
        self.relu1 = nn.LeakyReLU(0.3)
        self.inst1 = nn.InstanceNorm1d(16, device=device)
        self.drop1 = nn.Dropout(p=0.2)

        self.conv2 = nn.Conv1d(in_channels=16, out_channels=16, kernel_size=3, device=device)
        self.relu2 = nn.LeakyReLU(0.3)
        self.inst2 = nn.InstanceNorm1d(16, device=device)
        self.drop2 = nn.Dropout(p=0.2)

        self.conv3 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, stride=4, device=device)
        self.relu3 = nn.LeakyReLU(0.3)
        self.inst3 = nn.InstanceNorm1d(32, device=device)
        self.drop3 = nn.Dropout(p=0.2)

        self.conv4 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, stride=1, device=device)
        self.relu4 = nn.LeakyReLU(0.3)
        self.inst4 = nn.InstanceNorm1d(32, device=device)
        self.drop4 = nn.Dropout(p=0.2)

        self.conv5 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, stride=4, device=device)
        self.relu5 = nn.LeakyReLU(0.3)
        self.inst5 = nn.InstanceNorm1d(64, device=device)
        self.drop5 = nn.Dropout(p=0.2)

        self.conv6 = nn.Conv1d(in_channels=64, out_channels=100, kernel_size=5, stride=1, device=device)
        self.relu6 = nn.LeakyReLU(0.3)

        self.glap = nn.AvgPool1d(kernel_size=4)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.inst1(x)
        x = self.drop1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.inst2(x)
        x = self.drop2(x)

        x = self.conv3(x)
        x = self.relu3(x)
        x = self.inst3(x)
        x = self.drop3(x)

        x = self.conv4(x)
        x = self.relu4(x)
        x = self.inst4(x)
        x = self.drop4(x)
        
        x = self.conv5(x)
        x = self.relu5(x)
        x = self.inst5(x)
        x = self.drop5(x)
        
        x = self.conv6(x)
        x = self.relu6(x)

        x = self.glap(x)
        x = x.flatten(start_dim=1)

        logits = x
        return logits

In [7]:
class ChangClassifier(nn.Module):
    def __init__(self):
        super().__init__()

        self.densa = nn.Linear(in_features=100, out_features=8, device=device)

    def forward(self, x):
        logits = self.densa(x)
        return logits

In [307]:
enc = ChangEncoder()
cla = ChangClassifier()
logits = enc(X_batch)
logits = cla(logits)
logits

tensor([[-0.0674, -0.1977,  0.0196, -0.0244, -0.2632, -0.0230,  0.0354, -0.1145],
        [-0.3362, -0.0063,  0.0603, -0.1109, -0.1321, -0.0622,  0.1603, -0.0293],
        [ 0.0268,  0.0109,  0.0922, -0.2042, -0.0503,  0.0014, -0.1114, -0.2235],
        [ 0.0359, -0.0539,  0.0779, -0.1392, -0.2226, -0.2199,  0.0752, -0.0036],
        [-0.0097,  0.2022,  0.2664, -0.2262, -0.0377, -0.0352,  0.1647, -0.0462],
        [-0.0526,  0.2027,  0.3545, -0.0973, -0.0873, -0.0095,  0.0864, -0.0931],
        [ 0.0710,  0.0343,  0.0654, -0.2879,  0.0660,  0.0923,  0.1096, -0.0310],
        [-0.1459,  0.0090,  0.0919, -0.0597, -0.2078,  0.1630,  0.1305, -0.2380]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [288]:
opt1 = torch.optim.SGD(enc.parameters(), lr=0.1)
opt2 = torch.optim.SGD(cla.parameters(), lr=0.1)

In [308]:
opt = torch.optim.SGD(list(enc.parameters()) + list(cla.parameters()), lr=0.01)

In [151]:
labels = nn.functional.one_hot(y_batch.to(torch.int64), num_classes=8)

In [309]:
loss = nn.CrossEntropyLoss()
output = loss(logits, labels.double())
output

tensor(2.0258, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward1>)

# Treinamento

In [446]:
for posi in range(7):
    print(D[posi].iloc[:,:450].max().max(), D[posi].iloc[:,:450].min().min())

30.24330038394172 -27.905772938181507
29.10584118741573 -23.004128879086824
24.60838945941594 -23.373795119211223
27.10391661258158 -35.01022254108014
28.35246581033783 -32.08145329447701
25.498862803206386 -28.645988494886826
24.63936106304582 -26.89441535730718


In [447]:
posi = 0
# Vamos descartar a classe jump por ter poucas amostras
# E mudar o código de walking para 2 (era 7)
# aux = D[posi][D[posi]['activity code']!=2].replace({'activity code': 7}, 2)
# tarefas = np.array(['climbingdown', 'climbingup', 'walking', 'lying', 'running', 'sitting', 'standing'])
N = aux.shape[0]
X = aux.iloc[:,:450].values.reshape(N,3,150) / 30   # Normalizando para a faixa de -1 a 1 aproximadamente pela divisão por 30
y = aux.iloc[:,-3].values

In [448]:
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=1, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.1, random_state=1, stratify=y_train)

y_train = nn.functional.one_hot(torch.tensor(y_train), num_classes=8).to(torch.float32).to(device)
y_val = nn.functional.one_hot(torch.tensor(y_val), num_classes=8).to(torch.float32).to(device)
y_test = nn.functional.one_hot(torch.tensor(y_test), num_classes=8).to(torch.float32).to(device)

X_train = torch.tensor(X_train, dtype=torch.float32, device=device)
X_val = torch.tensor(X_val, dtype=torch.float32, device=device)
X_test = torch.tensor(X_test, dtype=torch.float32, device=device)

In [449]:
[y_train.shape, y_test.shape, y_val.shape]

[torch.Size([15881, 8]), torch.Size([4412, 8]), torch.Size([1765, 8])]

In [450]:
encoder = ChangEncoder().to(device)
classif = ChangClassifier().to(device)

# torch.manual_seed(42)

loss_fn = nn.CrossEntropyLoss()

optimizer = optim.Adam(list(encoder.parameters()) + list(classif.parameters()), lr=0.001)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

n_epochs = 20   # number of epochs to run
batch_size = 125  # size of each batch
batch_start = torch.arange(0, len(X_train), batch_size)

trainLoss = []
trainAcc = []
valLoss = []
valAcc = []

# training loop
for epoch in range(n_epochs):
    encoder.train()
    classif.train()
    with tqdm(batch_start, unit="batch", mininterval=0, disable=False) as bar:
        bar.set_description(f"Epoch {epoch}")
        for start in bar:
            # take a batch
            X_batch = X_train[start:start+batch_size,:,:]
            y_batch = y_train[start:start+batch_size,:]
            # forward pass
            y_pred = classif(encoder(X_batch))
            loss = loss_fn(y_pred, y_batch)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            # update weights
            optimizer.step()
            # print progress
            bar.set_postfix(loss=float(loss))
    # evaluate accuracy at end of each epoch
    encoder.eval()
    classif.eval()
    y_pred = classif(encoder(X_val))
    valLoss.append(float(loss_fn(y_pred, y_val)))
    labels = torch.nonzero(y_val)[:,1]
    labels_pred = torch.argmax(y_pred, dim=1)
    valAcc.append(multiclass_f1_score(labels_pred, labels, num_classes=8))
    y_pred = classif(encoder(X_train))
    trainLoss.append(float(loss_fn(y_pred, y_train)))
    labels = torch.nonzero(y_train)[:,1]
    labels_pred = torch.argmax(y_pred, dim=1)
    valAcc.append(multiclass_f1_score(labels_pred, labels, num_classes=8))

fig = go.Figure()

fig.add_trace(go.Scatter(y=valLoss, mode="lines", showlegend=True, name='validação'))
fig.add_trace(go.Scatter(y=trainLoss, mode="lines", showlegend=True, name='treino'))

fig.show()

Epoch 49: 100%|██████████| 128/128 [00:11<00:00, 11.04batch/s, loss=0.0689]


In [14]:
def treinoModelo(dados):
        N = dados.shape[0]
        X = dados.iloc[:,:450].values.reshape(N,3,150) / 30   # Normalizando para a faixa de -1 a 1 aproximadamente pela divisão por 30
        y = dados.iloc[:,-3].values
        X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=1, stratify=y)

        X_train, X_val, y_train, y_val = train_test_split(
                X_train, y_train, test_size=0.1, random_state=1, stratify=y_train)

        y_train = nn.functional.one_hot(torch.tensor(y_train), num_classes=8).to(torch.float32).to(device)
        y_val = nn.functional.one_hot(torch.tensor(y_val), num_classes=8).to(torch.float32).to(device)
        y_test = nn.functional.one_hot(torch.tensor(y_test), num_classes=8).to(torch.float32).to(device)

        X_train = torch.tensor(X_train, dtype=torch.float32, device=device)
        X_val = torch.tensor(X_val, dtype=torch.float32, device=device)
        X_test = torch.tensor(X_test, dtype=torch.float32, device=device)

        encoder = ChangEncoder().to(device)
        classif = ChangClassifier().to(device)
        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.Adam(list(encoder.parameters()) + list(classif.parameters()), lr=0.001)
        n_epochs = 20   # number of epochs to run
        batch_size = 125  # size of each batch
        batch_start = torch.arange(0, len(X_train), batch_size)
        trainLoss = []
        trainAcc = []
        valLoss = []
        valAcc = []

        # training loop
        for epoch in range(n_epochs):
                encoder.train()
                classif.train()
                with tqdm(batch_start, unit="batch", mininterval=0, disable=False) as bar:
                        bar.set_description(f"Epoch {epoch}")
                        for start in bar:
                                # take a batch
                                X_batch = X_train[start:start+batch_size,:,:]
                                y_batch = y_train[start:start+batch_size,:]
                                # forward pass
                                y_pred = classif(encoder(X_batch))
                                loss = loss_fn(y_pred, y_batch)
                                # backward pass
                                optimizer.zero_grad()
                                loss.backward()
                                # update weights
                                optimizer.step()
                                # print progress
                                bar.set_postfix(loss=float(loss))
                # evaluate accuracy at end of each epoch
                encoder.eval()
                classif.eval()
                y_pred = classif(encoder(X_val))
                valLoss.append(float(loss_fn(y_pred, y_val)))
                labels = torch.nonzero(y_val)[:,1]
                labels_pred = torch.argmax(y_pred, dim=1)
                valAcc.append(float(multiclass_f1_score(labels_pred, labels, num_classes=8)))
                y_pred = classif(encoder(X_train))
                trainLoss.append(float(loss_fn(y_pred, y_train)))
                labels = torch.nonzero(y_train)[:,1]
                labels_pred = torch.argmax(y_pred, dim=1)
                trainAcc.append(float(multiclass_f1_score(labels_pred, labels, num_classes=8)))

        fig = go.Figure()
        fig.add_trace(go.Scatter(y=valLoss, mode="lines", showlegend=True, name='validação'))
        fig.add_trace(go.Scatter(y=trainLoss, mode="lines", showlegend=True, name='treino'))
        fig.show()

        fig2 = go.Figure()
        fig2.add_trace(go.Scatter(y=valAcc, mode="lines", showlegend=True, name='validação'))
        fig2.add_trace(go.Scatter(y=trainAcc, mode="lines", showlegend=True, name='treino'))
        fig2.show()
        return encoder, classif


In [15]:
encoders = [None] * 7
classifs = [None] * 7
for posi in range(7):
    encoders[posi], classifs[posi] = treinoModelo(D[posi])

Epoch 19: 100%|██████████| 130/130 [00:02<00:00, 44.48batch/s, loss=0.622]


Epoch 19: 100%|██████████| 128/128 [00:07<00:00, 16.47batch/s, loss=0.769]


Epoch 19: 100%|██████████| 130/130 [00:07<00:00, 16.59batch/s, loss=0.774]


Epoch 19: 100%|██████████| 130/130 [00:07<00:00, 16.61batch/s, loss=0.488]


Epoch 19: 100%|██████████| 130/130 [00:07<00:00, 16.64batch/s, loss=0.686]


Epoch 19: 100%|██████████| 130/130 [00:07<00:00, 16.59batch/s, loss=0.5]  


Epoch 19: 100%|██████████| 130/130 [00:07<00:00, 16.34batch/s, loss=0.397]


# Experimentações

In [55]:
PATH = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\reprodução chang\\modelos\\'
for posi in range(7):
    encoders[posi].eval()
    classifs[posi].eval()
    torch.save(encoders[posi].state_dict(), PATH+'encoder'+str(posi)+'.pth')
    torch.save(classifs[posi].state_dict(), PATH+'classif'+str(posi)+'.pth')

In [13]:
PATH = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\reprodução chang\\modelos\\'
encoders = [None] * 7
classifs = [None] * 7
for posi in range(7):
    encoders[posi] = ChangEncoder()
    classifs[posi] = ChangClassifier()
    encoders[posi].load_state_dict(torch.load(PATH+'encoder'+str(posi)+'.pth'))
    classifs[posi].load_state_dict(torch.load(PATH+'classif'+str(posi)+'.pth'))
    encoders[posi].eval()
    classifs[posi].eval()

In [14]:
def testesDominios(dados):
    loss_fn = nn.CrossEntropyLoss()
    loss = np.zeros((7,8))
    acc = np.zeros((7,8))
    for posi in range(7):
        N = dados[posi].shape[0]
        X = dados[posi].iloc[:,:450].values.reshape(N,3,150) / 30   # Normalizando para a faixa de -1 a 1 aproximadamente pela divisão por 30
        y = dados[posi].iloc[:,-3].values
        X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=1, stratify=y)
        X_train, X_val, y_train, y_val = train_test_split(
                X_train, y_train, test_size=0.1, random_state=1, stratify=y_train)
        y_test = nn.functional.one_hot(torch.tensor(y_test), num_classes=8).to(torch.float32).to(device)
        X_test = torch.tensor(X_test, dtype=torch.float32, device=device)
        for i in range(7):
            print(i,posi)
            encoders[i].eval()
            classifs[i].eval()
            y_pred = classifs[i](encoders[i](X_test))
            labels = torch.nonzero(y_test)[:,1]
            labels_pred = torch.argmax(y_pred, dim=1)
            if i==posi:
                loss[i,0] = float(loss_fn(y_pred, y_test))
                acc[i,0] = float(multiclass_f1_score(labels_pred, labels, num_classes=8))
            else:
                loss[i,posi+1] = float(loss_fn(y_pred, y_test))
                acc[i,posi+1] = float(multiclass_f1_score(labels_pred, labels, num_classes=8))
    return loss, acc

In [15]:
loss, acc = testesDominios(D)

0 0
1 0
2 0
3 0
4 0
5 0
6 0
0 1
1 1
2 1
3 1
4 1
5 1
6 1
0 2
1 2
2 2
3 2
4 2
5 2
6 2
0 3
1 3
2 3
3 3
4 3
5 3
6 3
0 4
1 4
2 4
3 4
4 4
5 4
6 4
0 5
1 5
2 5
3 5
4 5
5 5
6 5
0 6
1 6
2 6
3 6
4 6
5 6
6 6


In [16]:
df = pd.DataFrame(acc)
df.columns = ['Fonte', 'chest', 'forearm', 'head', 'shin', 'thigh', 'upperarm', 'waist']
df.index = ['chest', 'forearm', 'head', 'shin', 'thigh', 'upperarm', 'waist']
df = df.round(2)
for i in range(7):
    df.iloc[i,i+1] = '-'

In [17]:
cols = df.columns.tolist()
ea = np.array(cols)[[0,3,1,6,2,7,5,4]].tolist()
df = df[ea]
linhas = df.index.tolist()
ea = np.array(linhas)[[2,0,5,1,6,4,3]]
df = df.reindex(ea)

In [24]:
latex_table = df3.to_latex(
    index=True,  # To not include the DataFrame index as a column in the table
    caption="Comparison of ML Model Performance Metrics",  # The caption to appear above the table in the LaTeX document
    label="tab:model_comparison",  # A label used for referencing the table within the LaTeX document
    position="htbp",  # The preferred positions where the table should be placed in the document ('here', 'top', 'bottom', 'page')
    column_format="l c|c c c c c c c",  # The format of the columns: left-aligned with vertical lines between them
    escape=False,  # Disable escaping LaTeX special characters in the DataFrame
    float_format="{:0.2f}".format  # Formats floats to two decimal places
)

print(latex_table)

\begin{table}[htbp]
\caption{Comparison of ML Model Performance Metrics}
\label{tab:model_comparison}
\begin{tabular}{l c|c c c c c c c}
\toprule
 & Fonte & head & chest & upperarm & forearm & waist & thigh & shin \\
\midrule
head & 0.09 & - & 0.13 & -0.01 & -0.14 & -0.28 & -0.09 & -0.07 \\
chest & 0.13 & -0.20 & - & -0.17 & -0.17 & -0.11 & -0.12 & -0.26 \\
upperarm & 0.06 & 0.03 & 0.00 & - & -0.18 & 0.03 & -0.13 & -0.15 \\
forearm & 0.10 & -0.29 & -0.29 & -0.40 & - & 0.06 & -0.29 & -0.30 \\
waist & 0.07 & -0.28 & -0.09 & -0.20 & -0.04 & - & -0.18 & -0.17 \\
thigh & 0.13 & -0.08 & -0.05 & -0.04 & -0.35 & -0.26 & - & -0.16 \\
shin & 0.06 & -0.10 & -0.12 & -0.12 & -0.28 & -0.07 & -0.14 & - \\
\bottomrule
\end{tabular}
\end{table}



In [19]:
table = np.matrix([[0.86, 0.00, 0.60, 0.51, 0.26, 0.14, 0.37, 0.23],
                   [0.94, 0.28, 0.00, 0.32, 0.25, 0.30, 0.42, 0.24],
                   [0.89, 0.47, 0.58, 0.00, 0.20, 0.18, 0.44, 0.35],
                   [0.88, 0.16, 0.12, 0.11, 0.00, 0.39, 0.12, 0.09],
                   [0.93, 0.16, 0.22, 0.05, 0.25, 0.00, 0.15, 0.02],
                   [0.94, 0.39, 0.39, 0.45, 0.05, 0.16, 0.00, 0.40],
                   [0.92, 0.06, 0.29, 0.33, 0.03, 0.09, 0.30, 0.00]])

In [20]:
df2 = pd.DataFrame(table)
df2.columns = ['Fonte', 'head', 'chest', 'upperarm', 'forearm', 'waist', 'thigh', 'shin']
df2.index = ['head', 'chest', 'upperarm', 'forearm', 'waist', 'thigh', 'shin']
for i in range(7):
    df2.iloc[i,i+1] = '-'
df2

,Fonte,head,chest,upperarm,forearm,waist,thigh,shin
head,0.86,-,0.6,0.51,0.26,0.14,0.37,0.23
chest,0.94,0.28,-,0.32,0.25,0.3,0.42,0.24
upperarm,0.89,0.47,0.58,-,0.2,0.18,0.44,0.35
forearm,0.88,0.16,0.12,0.11,-,0.39,0.12,0.09
waist,0.93,0.16,0.22,0.05,0.25,-,0.15,0.02
thigh,0.94,0.39,0.39,0.45,0.05,0.16,-,0.4
shin,0.92,0.06,0.29,0.33,0.03,0.09,0.3,-


In [23]:
vals = df.values
for i in range(7):
    vals[i,i+1] = 0.
diff = np.array(table-vals).astype(float)
df3 = pd.DataFrame(diff)
df3.columns = ['Fonte', 'head', 'chest', 'upperarm', 'forearm', 'waist', 'thigh', 'shin']
df3.index = ['head', 'chest', 'upperarm', 'forearm', 'waist', 'thigh', 'shin']
for i in range(7):
    df3.iloc[i,i+1] = '-'
df3

,Fonte,head,chest,upperarm,forearm,waist,thigh,shin
head,0.09,-,0.13,-0.01,-0.14,-0.28,-0.09,-0.07
chest,0.13,-0.2,-,-0.17,-0.17,-0.11,-0.12,-0.26
upperarm,0.06,0.03,0.0,-,-0.18,0.03,-0.13,-0.15
forearm,0.10,-0.29,-0.29,-0.4,-,0.06,-0.29,-0.3
waist,0.07,-0.28,-0.09,-0.2,-0.04,-,-0.18,-0.17
thigh,0.13,-0.08,-0.05,-0.04,-0.35,-0.26,-,-0.16
shin,0.06,-0.1,-0.12,-0.12,-0.28,-0.07,-0.14,-


# Treinamento de adaptação

In [69]:
dFont = D[0]
dAlvo = D[1]

N = 22100   # Vamos usar esse número mínimo de amostras por domínio
X = dFont.iloc[:N,:450].values.reshape(N,3,150) / 30   # Normalizando para a faixa de -1 a 1 aproximadamente pela divisão por 30
y = dFont.iloc[:N,-3].values
X_train1, _, y_train1, _ = train_test_split(
    X, y, test_size=0.2, random_state=1, stratify=y)

X_train1, X_val1, y_train1, y_val1 = train_test_split(
    X_train1, y_train1, test_size=0.1, random_state=1, stratify=y_train1)
    
X = dAlvo.iloc[:N,:450].values.reshape(N,3,150) / 30   # Normalizando para a faixa de -1 a 1 aproximadamente pela divisão por 30
y = dAlvo.iloc[:N,-3].values
X_train2, _, y_train2, _ = train_test_split(
    X, y, test_size=0.2, random_state=1, stratify=y)

X_train2, X_val2, y_train2, y_val2 = train_test_split(
    X_train2, y_train2, test_size=0.1, random_state=1, stratify=y_train2)

y_train1 = nn.functional.one_hot(torch.tensor(y_train1), num_classes=8).to(torch.float32).to(device)
y_val1 = nn.functional.one_hot(torch.tensor(y_val1), num_classes=8).to(torch.float32).to(device)
X_train1 = torch.tensor(X_train1, dtype=torch.float32, device=device)
X_val1 = torch.tensor(X_val1, dtype=torch.float32, device=device)

y_train2 = nn.functional.one_hot(torch.tensor(y_train2), num_classes=8).to(torch.float32).to(device)
y_val2 = nn.functional.one_hot(torch.tensor(y_val2), num_classes=8).to(torch.float32).to(device)
X_train2 = torch.tensor(X_train2, dtype=torch.float32, device=device)
X_val2 = torch.tensor(X_val2, dtype=torch.float32, device=device)

In [9]:
def mmd2u(x, y, c):
    n = x.shape[0]
    m = y.shape[0]
    xy = torch.vstack((x,y))
    dists = torch.cdist(xy, xy)
    k = torch.exp( (-1/(2*c)) * dists**2 )
    k_x = torch.triu(k[:n, :n], diagonal=1)
    k_y = torch.triu(k[n:, n:], diagonal=1)
    k_xy = k[:n, n:]
    mmd = 2*k_x.sum()/(n*(n-1)) + 2*k_y.sum()/(m*(m-1)) - 2*k_xy.sum()/(n*m)
    return mmd

In [189]:
emb1 = torch.randn(5,2)
emb2 = torch.randn(5,2)
print(mmd2u(emb2,emb1,1))
fig = px.scatter(x=emb1[:,0].cpu().detach().numpy(), y=emb1[:,1].cpu().detach().numpy(), width=700, height=500)
fig.add_trace(go.Scatter(x=emb2[:,0].cpu().detach().numpy(), y=emb2[:,1].cpu().detach().numpy(), mode='markers', showlegend=True))

tensor(-0.1854)


In [10]:
def sigma_median(xy):
    dists = nn.functional.pdist(xy)
    return torch.median(dists)

In [11]:
class mmdLoss(nn.Module):
    def __init__(self):
        super(mmdLoss, self).__init__()

    def forward(self, x, y, c):
        n = x.shape[0]
        m = y.shape[0]
        xy = torch.vstack((x,y))
        dists = torch.cdist(xy, xy)
        k = torch.exp( (-1/(2*c)) * dists**2 )
        k_x = torch.triu(k[:n, :n], diagonal=1)
        k_y = torch.triu(k[n:, n:], diagonal=1)
        k_xy = k[:n, n:]
        mmd = 2*k_x.sum()/(n*(n-1)) + 2*k_y.sum()/(m*(m-1)) - 2*k_xy.sum()/(n*m)
        return mmd

In [71]:
emb1 = encoders[0](X_val1)
emb2 = encoders[0](X_val2)

In [76]:
torch.cuda.empty_cache()

In [75]:
sigma = sigma_median(torch.vstack((emb1,emb2)))
c = sigma ** 2
print(c)

tensor(260.0161, device='cuda:0', grad_fn=<PowBackward0>)


In [206]:
vals = [0.01, 0.1, 1, 2, 3, 4, 5, 6, 10, 20, 50, 100, 200, 250, 300, 400, 500, 1000]
aux = np.zeros_like(vals)
for i, j in enumerate(vals):
    aux[i] = mmd2u(emb1,emb2,j).cpu().detach().numpy()

In [210]:
px.line(x=vals,y=aux, markers='lines+markers')

In [88]:
def treinoAdapt(dFont, dAlvo):
    N = 22100   # Vamos usar esse número mínimo de amostras por domínio
    X = dFont.iloc[:N,:450].values.reshape(N,3,150) / 30   # Normalizando para a faixa de -1 a 1 aproximadamente pela divisão por 30
    y = dFont.iloc[:N,-3].values
    X_train1, _, y_train1, _ = train_test_split(
        X, y, test_size=0.2, random_state=1, stratify=y)
    X_train1, X_val1, y_train1, y_val1 = train_test_split(
        X_train1, y_train1, test_size=0.1, random_state=1, stratify=y_train1)
        
    X = dAlvo.iloc[:N,:450].values.reshape(N,3,150) / 30   # Normalizando para a faixa de -1 a 1 aproximadamente pela divisão por 30
    y = dAlvo.iloc[:N,-3].values
    X_train2, _, y_train2, _ = train_test_split(
        X, y, test_size=0.2, random_state=1, stratify=y)
    X_train2, X_val2, y_train2, y_val2 = train_test_split(
        X_train2, y_train2, test_size=0.1, random_state=1, stratify=y_train2)

    y_train1 = nn.functional.one_hot(torch.tensor(y_train1), num_classes=8).to(torch.float32).to(device)
    y_val1 = nn.functional.one_hot(torch.tensor(y_val1), num_classes=8).to(torch.float32).to(device)
    X_train1 = torch.tensor(X_train1, dtype=torch.float32, device=device).detach()
    X_val1 = torch.tensor(X_val1, dtype=torch.float32, device=device).detach()
    
    y_train2 = nn.functional.one_hot(torch.tensor(y_train2), num_classes=8).to(torch.float32).to(device)
    y_val2 = nn.functional.one_hot(torch.tensor(y_val2), num_classes=8).to(torch.float32).to(device)
    X_train2 = torch.tensor(X_train2, dtype=torch.float32, device=device).detach()
    X_val2 = torch.tensor(X_val2, dtype=torch.float32, device=device).detach()

    encoder = ChangEncoder().to(device)
    classif = ChangClassifier().to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer1 = optim.Adam(list(encoder.parameters()) + list(classif.parameters()), lr=0.001)
    optimizer2 = optim.Adam(encoder.parameters(), lr=0.001)
    
    n_epochs = 20   # number of epochs to run
    batch_size = 125  # size of each batch
    batch_start = torch.arange(0, len(X_train1), batch_size)
    trainLoss = []
    trainAcc = []
    valLoss = []
    valAcc = []

    for epoch in range(n_epochs):
        encoder.train()
        classif.train()
        with tqdm(batch_start, unit="batch", mininterval=0, disable=False) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                X_batch1 = X_train1[start:start+batch_size,:,:]
                X_batch2 = X_train2[start:start+batch_size,:,:]
                y_batch1 = y_train1[start:start+batch_size,:]
                # Passo de treinamento com loss de classificação
                y_pred = classif(encoder(X_batch1))
                loss1 = loss_fn(y_pred, y_batch1)
                optimizer1.zero_grad()
                loss1.backward()
                optimizer1.step()
                # Passo de treinamento com loss mmd
                emb1 = encoder(X_batch1)
                emb2 = encoder(X_batch2)
                loss2 = 0
                for i in vals: 
                    loss2 += mmd2u(emb1,emb2,i)
                optimizer2.zero_grad()
                loss2.backward()
                optimizer2.step()
                # print progress
                bar.set_postfix(loss=float(loss1))
        encoder.eval()
        classif.eval()
        y_pred = classif(encoder(X_val1))
        valLoss.append(float(loss_fn(y_pred, y_val1)))
        labels = torch.nonzero(y_val1)[:,1]
        labels_pred = torch.argmax(y_pred, dim=1)
        valAcc.append(float(multiclass_f1_score(labels_pred, labels, num_classes=8)))
        y_pred = classif(encoder(X_train1))
        trainLoss.append(float(loss_fn(y_pred, y_train1)))
        labels = torch.nonzero(y_train1)[:,1]
        labels_pred = torch.argmax(y_pred, dim=1)
        trainAcc.append(float(multiclass_f1_score(labels_pred, labels, num_classes=8)))

    fig = go.Figure()
    fig.add_trace(go.Scatter(y=valLoss, mode="lines", showlegend=True, name='validação'))
    fig.add_trace(go.Scatter(y=trainLoss, mode="lines", showlegend=True, name='treino'))
    fig.show()

    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(y=valAcc, mode="lines", showlegend=True, name='validação'))
    fig2.add_trace(go.Scatter(y=trainAcc, mode="lines", showlegend=True, name='treino'))
    fig2.show()
    return encoder, classif


In [89]:
encoder, classif = treinoAdapt(dFont, dAlvo)

Epoch 19: 100%|██████████| 128/128 [00:13<00:00,  9.33batch/s, loss=0.572]


In [90]:
PATH = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\reprodução chang\\modelos\\'
encoder.eval()
classif.eval()
torch.save(encoder.state_dict(), PATH+'encoderAdpt.pth')
torch.save(classif.state_dict(), PATH+'classifAdpt.pth')

In [92]:
encoder = ChangEncoder()
classif = ChangClassifier()
encoder.load_state_dict(torch.load(PATH+'encoderAdpt.pth'))
classif.load_state_dict(torch.load(PATH+'classifAdpt.pth'))
encoder.eval()
classif.eval()

ChangClassifier(
  (densa): Linear(in_features=100, out_features=8, bias=True)
)

# Treinamento com DataLoader

In [9]:
posi = 0
N = D[posi].shape[0]
X = D[posi].iloc[:,:450].values.reshape(N,3,150) / 30   # Normalizando para a faixa de -1 a 1 aproximadamente pela divisão por 30
y = D[posi].iloc[:,-3].values

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=1, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.1, random_state=1, stratify=y_train)

y_train = nn.functional.one_hot(torch.tensor(y_train), num_classes=8).to(torch.float32).to(device)
y_val = nn.functional.one_hot(torch.tensor(y_val), num_classes=8).to(torch.float32).to(device)
y_test = nn.functional.one_hot(torch.tensor(y_test), num_classes=8).to(torch.float32).to(device)

X_train = torch.tensor(X_train, dtype=torch.float32, device=device)
X_val = torch.tensor(X_val, dtype=torch.float32, device=device)
X_test = torch.tensor(X_test, dtype=torch.float32, device=device)

In [14]:
trainset = RealWorldDataset(X_train, y_train)
batch_size = 125
trainLoader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

In [16]:
encoder = ChangEncoder().to(device)
classif = ChangClassifier().to(device)

loss_fn = nn.CrossEntropyLoss()

optimizer = optim.Adam(list(encoder.parameters()) + list(classif.parameters()), lr=0.001)

n_epochs = 20   # number of epochs to run
trainLoss = []
trainAcc = []
valLoss = []
valAcc = []

for epoch in tqdm(range(n_epochs)):
    encoder.train()
    classif.train()
    for X_batch, y_batch in trainLoader:
        y_pred = classif(encoder(X_batch))
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # evaluate accuracy at end of each epoch
    encoder.eval()
    classif.eval()
    y_pred = classif(encoder(X_val))
    valLoss.append(float(loss_fn(y_pred, y_val)))
    labels = torch.nonzero(y_val)[:,1]
    labels_pred = torch.argmax(y_pred, dim=1)
    valAcc.append(multiclass_f1_score(labels_pred, labels, num_classes=8))
    y_pred = classif(encoder(X_train))
    trainLoss.append(float(loss_fn(y_pred, y_train)))
    labels = torch.nonzero(y_train)[:,1]
    labels_pred = torch.argmax(y_pred, dim=1)
    valAcc.append(multiclass_f1_score(labels_pred, labels, num_classes=8))

100%|██████████| 20/20 [00:32<00:00,  1.60s/it]


In [17]:
fig = go.Figure()

fig.add_trace(go.Scatter(y=valLoss, mode="lines", showlegend=True, name='validação'))
fig.add_trace(go.Scatter(y=trainLoss, mode="lines", showlegend=True, name='treino'))

fig.show()

In [ ]:
encoder = ChangEncoder().to(device)
classif = ChangClassifier().to(device)

# torch.manual_seed(42)

loss_fn = nn.CrossEntropyLoss()

optimizer = optim.Adam(list(encoder.parameters()) + list(classif.parameters()), lr=0.001)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

n_epochs = 20   # number of epochs to run

trainLoss = []
trainAcc = []
valLoss = []
valAcc = []

# training loop
for epoch in range(n_epochs):
    encoder.train()
    classif.train()
    with tqdm(batch_start, unit="batch", mininterval=0, disable=False) as bar:
        bar.set_description(f"Epoch {epoch}")
        for start in bar:
            # take a batch
            X_batch = X_train[start:start+batch_size,:,:]
            y_batch = y_train[start:start+batch_size,:]
            # forward pass
            y_pred = classif(encoder(X_batch))
            loss = loss_fn(y_pred, y_batch)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            # update weights
            optimizer.step()
            # print progress
            bar.set_postfix(loss=float(loss))
    # evaluate accuracy at end of each epoch
    encoder.eval()
    classif.eval()
    y_pred = classif(encoder(X_val))
    valLoss.append(float(loss_fn(y_pred, y_val)))
    labels = torch.nonzero(y_val)[:,1]
    labels_pred = torch.argmax(y_pred, dim=1)
    valAcc.append(multiclass_f1_score(labels_pred, labels, num_classes=8))
    y_pred = classif(encoder(X_train))
    trainLoss.append(float(loss_fn(y_pred, y_train)))
    labels = torch.nonzero(y_train)[:,1]
    labels_pred = torch.argmax(y_pred, dim=1)
    valAcc.append(multiclass_f1_score(labels_pred, labels, num_classes=8))

fig = go.Figure()

fig.add_trace(go.Scatter(y=valLoss, mode="lines", showlegend=True, name='validação'))
fig.add_trace(go.Scatter(y=trainLoss, mode="lines", showlegend=True, name='treino'))

fig.show()

# Projeções com t-SNE e umap

In [27]:
from sklearn.manifold import TSNE
import umap

In [80]:
posi = 0
N = D[posi].shape[0]
X = D[posi].iloc[:,:450].values.reshape(N,3,150) / 30   # Normalizando para a faixa de -1 a 1 aproximadamente pela divisão por 30
y = D[posi].iloc[:,-3].values
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=1, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.1, random_state=1, stratify=y_train)
# y_test = nn.functional.one_hot(torch.tensor(y_test), num_classes=8).to(torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32, device=device)

In [93]:
emb = encoder(X_test)
visu = TSNE(n_components=2, perplexity=10).fit_transform(emb.cpu().detach().numpy())

In [94]:
inds = np.argsort(y_test)
fig = px.scatter(x=visu[inds,0],y=visu[inds,1], color=tarefas[y_test[inds]], width=1000, height=500)
fig.update_layout(xaxis_range=[-100,100])
fig.update_layout(yaxis_range=[-100,100])
fig.show()

In [68]:
posicao

array(['chest', 'forearm', 'head', 'shin', 'thigh', 'upperarm', 'waist'],
      dtype='<U8')